In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.applications.imagenet_utils import decode_predictions
import numpy as np
import cv2
from cv2 import *
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
from scipy.misc import toimage
import sys
sys.path.append("..")
from src.network.GoogleLenet import GoogLeNet

image_width = 75
image_height = 160
model_g = GoogLeNet.build(image_height, image_width, 3, 61)
model_g.summary()

ImageNet...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 75, 3)   0                                            
__________________________________________________________________________________________________
conv1_7x7_s2 (Conv2D)           (None, 80, 38, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_7x7_s2_bn (BatchNormaliza (None, 80, 38, 64)   256         conv1_7x7_s2[0][0]               
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 40, 19, 64)   0           conv1_7x7_s2_bn[0][0]            
_________________________________________________________________________________________________

In [4]:
for layer in model_g.layers:
    print(layer.name)

input_1
conv1_7x7_s2
conv1_7x7_s2_bn
max_pooling2d_1
conv2_3x3
conv2_3x3_bn
max_pooling2d_2
inception_3a_3x3_reduce
inception_3a5x5_reduce
inception_3a_3x3_reduce_bn
inception_3a5x5_reduce_bn
max_pooling2d_3
inception_3a_1x1
inception_3a_3x3
inception_3a_5x5
inception_3a_pool_proj
inception_3a_1x1_bn
inception_3a_3x3_bn
inception_3a_5x5_bn
inception_3a_pool_proj_bn
concatenate_1
inception_3b_3x3_reduce
inception_3b5x5_reduce
inception_3b_3x3_reduce_bn
inception_3b5x5_reduce_bn
max_pooling2d_4
inception_3b_1x1
inception_3b_3x3
inception_3b_5x5
inception_3b_pool_proj
inception_3b_1x1_bn
inception_3b_3x3_bn
inception_3b_5x5_bn
inception_3b_pool_proj_bn
concatenate_2
max_pooling2d_5
inception_4a_3x3_reduce
inception_4a5x5_reduce
inception_4a_3x3_reduce_bn
inception_4a5x5_reduce_bn
max_pooling2d_6
inception_4a_1x1
inception_4a_3x3
inception_4a_5x5
inception_4a_pool_proj
inception_4a_1x1_bn
inception_4a_3x3_bn
inception_4a_5x5_bn
inception_4a_pool_proj_bn
concatenate_3
inception_4b_3x3_reduc

In [12]:
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

def model_from_layername(model, layer_name):
    model_extractfeatures = Model(inputs=model.input, outputs=model.get_layer(name=layer_name).output)
    x = model_extractfeatures.output
    x = GlobalAveragePooling2D()(x)
    #x = Dense(1024, activation = 'relu')(x)
    predictions = Dense(1, activation = 'sigmoid')(x)
    model_ret = Model(inputs=model_extractfeatures.input, outputs=predictions)
    for layer in model_ret.layers[:-2]:
        print(layer.name, ":", layer.trainable)
        layer.trainable = False
    return model_ret

model_low = model_from_layername(model_g, "concatenate_3")
model_low.summary()
for layer in model_low.layers:
    print(layer.name, ":", layer.trainable)
print("----------------------------------------------")
    
model_mid = model_from_layername(model_g, "concatenate_6")
model_mid.summary()
for layer in model_mid.layers:
    print(layer.name, ":", layer.trainable)
print("----------------------------------------------")
    
model_hig = model_from_layername(model_g, "concatenate_9")
model_hig.summary()
for layer in model_hig.layers:
    print(layer.name, ":", layer.trainable)
print("----------------------------------------------")

input_1 : False
conv1_7x7_s2 : False
conv1_7x7_s2_bn : False
max_pooling2d_1 : False
conv2_3x3 : False
conv2_3x3_bn : False
max_pooling2d_2 : False
inception_3a_3x3_reduce : False
inception_3a5x5_reduce : False
inception_3a_3x3_reduce_bn : False
inception_3a5x5_reduce_bn : False
max_pooling2d_3 : False
inception_3a_1x1 : False
inception_3a_3x3 : False
inception_3a_5x5 : False
inception_3a_pool_proj : False
inception_3a_1x1_bn : False
inception_3a_3x3_bn : False
inception_3a_5x5_bn : False
inception_3a_pool_proj_bn : False
concatenate_1 : False
inception_3b_3x3_reduce : False
inception_3b5x5_reduce : False
inception_3b_3x3_reduce_bn : False
inception_3b5x5_reduce_bn : False
max_pooling2d_4 : False
inception_3b_1x1 : False
inception_3b_3x3 : False
inception_3b_5x5 : False
inception_3b_pool_proj : False
inception_3b_1x1_bn : False
inception_3b_3x3_bn : False
inception_3b_5x5_bn : False
inception_3b_pool_proj_bn : False
concatenate_2 : False
max_pooling2d_5 : False
inception_4a_3x3_reduce 

inception_3a_1x1 : False
inception_3a_3x3 : False
inception_3a_5x5 : False
inception_3a_pool_proj : False
inception_3a_1x1_bn : False
inception_3a_3x3_bn : False
inception_3a_5x5_bn : False
inception_3a_pool_proj_bn : False
concatenate_1 : False
inception_3b_3x3_reduce : False
inception_3b5x5_reduce : False
inception_3b_3x3_reduce_bn : False
inception_3b5x5_reduce_bn : False
max_pooling2d_4 : False
inception_3b_1x1 : False
inception_3b_3x3 : False
inception_3b_5x5 : False
inception_3b_pool_proj : False
inception_3b_1x1_bn : False
inception_3b_3x3_bn : False
inception_3b_5x5_bn : False
inception_3b_pool_proj_bn : False
concatenate_2 : False
max_pooling2d_5 : False
inception_4a_3x3_reduce : False
inception_4a5x5_reduce : False
inception_4a_3x3_reduce_bn : False
inception_4a5x5_reduce_bn : False
max_pooling2d_6 : False
inception_4a_1x1 : False
inception_4a_3x3 : False
inception_4a_5x5 : False
inception_4a_pool_proj : False
inception_4a_1x1_bn : False
inception_4a_3x3_bn : False
inception_

In [14]:
model_low.save("../results/models/model_low.hdf5")
model_mid.save("../results/models/model_mid.hdf5")
model_hig.save("../results/models/model_hig.hdf5")

In [15]:
from keras.models import load_model
model_test = load_model("../results/models/model_low.hdf5")
model_test.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 75, 3)   0                                            
__________________________________________________________________________________________________
conv1_7x7_s2 (Conv2D)           (None, 80, 38, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_7x7_s2_bn (BatchNormaliza (None, 80, 38, 64)   256         conv1_7x7_s2[0][0]               
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 40, 19, 64)   0           conv1_7x7_s2_bn[0][0]            
__________________________________________________________________________________________________
conv2_3x3 

/home/anhaoran/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
